In [1]:
import pandas as pd
house = pd.read_csv("./boston_house.csv")
house.head(3)

,AGE,B,RM,CRIM,DIS,INDUS,LSTAT,NOX,PTRATIO,RAD,ZN,TAX,CHAS,Target
0,65.2,396.90,6.575,0.00632,4.0900,2.31,4.98,0.538,15.3,1,18.0,296,0,24.0
1,78.9,396.90,6.421,0.02731,4.9671,7.07,9.14,0.469,17.8,2,0.0,242,0,21.6
2,61.1,392.83,7.185,0.02729,4.9671,7.07,4.03,0.469,17.8,2,0.0,242,0,34.7


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

house = pd.read_csv("./boston_house.csv")
X_train, X_test, y_train, y_test = train_test_split(house.iloc[:,:-1], house.iloc[:, -1], test_size = 0.2)

[문제]보스턴 집값 데이터(boston_house.csv)를 사용하여 아래 내용을 구현하시오.
<처리조건>
학습 데이터 : 80%
테스트 데이터 : 20%

(1) 직접 구현한 손실함수와 수치미분 함수를 활용하여 집값을 예측는 예제를 작성하시오.

(2) 사이킷런이 제공하는 LinearRegression을 사용하여 집값을 예측하는 예제를 작성하시오.

In [7]:
# [1번]
import numpy as np
from sklearn.model_selection import train_test_split 

house = pd.read_csv("./boston_house.csv")
x_data = house.iloc[:, :-1]     
t_data = house.iloc[:, [-1]]

x_train, x_test, t_train, t_test = train_test_split(x_data, t_data, test_size=0.2)

w = np.random.rand(x_train.shape[1], 1)
b = np.random.rand(1)

def loss_function(x, t):
    y = np.dot(x, w) + b
    return np.mean((t - y) ** 2)


def numerical_derivative(f, x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=["multi_index"], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]

        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x_train)  

        x[idx] = float(tmp_val) - delta_x
        fx2 = f(x_train)

        grad[idx] = (fx1 - fx2) / (2 * delta_x)
        x[idx] = tmp_val
        it.iternext()

    return grad


def predict(x):
    return np.dot(x, w) + b

#----------------------------------------------------------------------#
learning_rate = 1e-6

f = lambda x: loss_function(x_train, t_train)

for step in range(10001):
    w -= learning_rate * numerical_derivative(f, w)
    b -= learning_rate * numerical_derivative(f, b)

    # if step % 1000 == 0:
    #     print("step =", step,
    #           "train loss =", loss_function(x_train, t_train),
    #           "가중치 =", w.T,
    #           "절편 =", b)


test_loss = loss_function(x_test, t_test)
print("테스트 데이터 최종 성능 평가 =", test_loss)


print("Test 예측값:", predict(x_test[:5]))
print("Test 실제값:", t_test[:5])


Initial loss (train) = 565598.7908435722
테스트 데이터 최종 성능 평가 = 85.28223413566425
Test 예측값: [[25.75791168]
 [21.73421983]
 [26.6991927 ]
 [24.9926193 ]
 [21.20236467]]
Test 실제값:      Target
280    45.4
464    21.4
271    25.2
163    50.0
377    13.3


In [12]:
#[2번]
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

house = pd.read_csv("./boston_house.csv")

x_data = house.iloc[:, :-1]   
t_data = house.iloc[:, [-1]]  

x_train, x_test, t_train, t_test = train_test_split(x_data, t_data, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(x_train, t_train)

print("가중치(coef):\n", model.coef_)
print("절편(intercept):", model.intercept_)

t_pred = model.predict(x_test)

print("예측값:\n", t_pred[:5])
print("실제값:\n", t_test[:5].values)


가중치(coef):
 [[-6.29636221e-03  1.23513347e-02  4.43883520e+00 -1.13055924e-01
  -1.44786537e+00  4.03807204e-02 -5.08571424e-01 -1.72026334e+01
  -9.15456240e-01  2.62429736e-01  3.01104641e-02 -1.06467863e-02
   2.78443820e+00]]
절편(intercept): [30.24675099]
예측값:
 [[28.99672362]
 [36.02556534]
 [14.81694405]
 [25.03197915]
 [18.76987992]]
실제값:
 [[23.6]
 [32.4]
 [13.6]
 [22.8]
 [16.1]]
